**0. Code for Colab Debugging**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/lxmert/data/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/lxmert/data


**1-1. Run Depth First Search on KG (Node only)**. Root is an admission node.

In [2]:
from tqdm import tqdm
import pickle
import torch
import numpy as np
import os

NUM_SPECIAL_TOKENS = 2

def get_childs(subgraph, depth, heads):
    temp_seq = list()
    for head in heads:
        temp_seq += subgraph[depth][head]
    return temp_seq

triples = [x.split() for x in open('train2id.txt').read().splitlines()[1:]]
node2edge = {(h,t):r for h,t,r in triples}
nodes = {' '.join(x.split()[:-1]):x.split()[-1] for x in open('entity2id.txt').read().splitlines()[1:]}
literals = {k:int(v)+NUM_SPECIAL_TOKENS for (k,v) in list(nodes.items()) if '^^' in k}
edges = {x.split()[0]:x.split()[1] for x in open('relation2id.txt').read().splitlines()[1:]}

# Extract Admission Nodes & Literals
adm_node = list()
for node in list(nodes.items()):
    if 'hadm' in node[0]:
        adm_node.append(node[1])
        
# Initialize subgraph
subgraph_norel = [{node:list() for node in adm_node}]

#subgraph_rel = dict(adm_node)
#node_dict = list(subgraph_norel.keys())

# Depth First Search
print('start preprocessing')
level = 0
while len(triples)>0:
    queue = list()
    print('level:{}'.format(level))
    for triple in tqdm(triples):
        if triple[0] in subgraph_norel[level]:
            subgraph_norel[level][triple[0]].append(triple[1])
            flag = False
        else:
            flag = True
        if flag:
            queue.append(triple)
    print('{}/{}'.format(len(queue),len(triples)))
    new_head = list()
    for heads in list(subgraph_norel[level].values()):
        new_head+=heads
    subgraph_norel.append({k:list() for k in new_head})
    triples = queue
    level += 1

# Build subgraph
subgraphs = dict()
max_len = 900
cnt = 0
for head in tqdm(list(subgraph_norel[0].keys())):
    depth=0
    seq = [head]
    heads = [head]
    while depth<level:
        heads = get_childs(subgraph_norel,depth,heads)
        seq += heads
        depth+=1
    if len(seq)<=max_len:
        subgraphs[head]=[int(x)+NUM_SPECIAL_TOKENS for x in seq]+[0]*(max_len-len(seq))

# Align subgraph and note
aid = [nodes['</hadm_id/{}>'.format(x)] for x in open('p_hadm_ids.txt').read().splitlines() if (len(x)>0) and ('</hadm_id/{}>'.format(x) in nodes)]
note = [x for x in open('p_sections.txt').read().splitlines() if (len(x)>0)]
note_aid_pair = [(x,y) for (x,y) in zip(aid,note) if x in subgraphs]
print('{}/{}'.format(len(aid),len(adm_node)))
print(len(note))
print(len(note_aid_pair))

  1%|          | 117834/18755503 [00:00<00:15, 1178327.08it/s]

start preprocessing
level:0


100%|██████████| 18755503/18755503 [00:13<00:00, 1434326.22it/s]


15929096/18755503


  0%|          | 55333/15929096 [00:00<00:28, 553326.57it/s]

level:1


100%|██████████| 15929096/15929096 [00:29<00:00, 544815.56it/s]


0/15929096


100%|██████████| 31969/31969 [00:08<00:00, 3647.96it/s]


32202/31969
32215
27075


**1-2. Run Depth First Search on KG (Node & Relation)**. Root is an admission node.

In [ ]:
from tqdm import tqdm
import pickle
import torch
import numpy as np
import os

NUM_SPECIAL_TOKENS = 2

def get_childs_withrel(subgraph, depth, heads,node2edge):
    temp_seq = list()
    temp_heads = list()
    for head in heads:
        node_set = [(head,tail) for tail in subgraph[depth][head]]
        for node_pair in node_set:
            temp_seq += ['r'+node2edge[node_pair],node_pair[1]]
        temp_heads += subgraph[depth][head]
    return temp_seq, temp_heads

triples = [x.split() for x in open('train2id.txt').read().splitlines()[1:]]
node2edge = {(h,t):r for h,t,r in triples}
nodes = {' '.join(x.split()[:-1]):x.split()[-1] for x in open('entity2id.txt').read().splitlines()[1:]}
literals = {k:int(v)+NUM_SPECIAL_TOKENS for (k,v) in list(nodes.items()) if '^^' in node[0]}
edges = {x.split()[0]:x.split()[1] for x in open('relation2id.txt').read().splitlines()[1:]}

# Extract Admission Nodes & Literals
adm_node = list()
for node in list(nodes.items()):
    if 'hadm' in node[0]:
        adm_node.append(node[1])   
        
# Initialize subgraph
subgraph_norel = [{node:list() for node in adm_node}]

#subgraph_rel = dict(adm_node)
#node_dict = list(subgraph_norel.keys())

# Depth First Search
print('start preprocessing')
level = 0
while len(triples)>0:
    queue = list()
    print('level:{}'.format(level))
    for triple in tqdm(triples):
        if triple[0] in subgraph_norel[level]:
            subgraph_norel[level][triple[0]].append(triple[1])
            flag = False
        else:
            flag = True
        if flag:
            queue.append(triple)
    print('{}/{}'.format(len(queue),len(triples)))
    new_head = list()
    for heads in list(subgraph_norel[level].values()):
        new_head+=heads
    subgraph_norel.append({k:list() for k in new_head})
    triples = queue
    level += 1

# Build subgraph
subgraphs = list()
max_len = 0
for head in tqdm(list(subgraph_norel[0].keys())):
    depth=0
    seq = [head]
    heads = [head]
    while depth<level:
        seqs, heads = get_childs_withrel(subgraph_norel,depth,heads,node2edge)
        seq += seqs
        depth+=1
    subgraphs.append(list(map(lambda x: int(x)+NUM_SPECIAL_TOKENS if 'r' not in x else -(int(x.split('r')[-1])+1),seq)))
    if len(seq)>max_len:
        max_len = len(seq)


# Align subgraph and note
aid = [nodes['</hadm_id/{}>'.format(x)] for x in open('p_hadm_ids.txt').read().splitlines() if (len(x)>0) and ('</hadm_id/{}>'.format(x) in nodes)]
note = [x for x in open('p_sections.txt').read().splitlines() if (len(x)>0)]
note_aid_pair = [(x,y) for (x,y) in zip(aid,note) if x in subgraphs]
print('{}/{}'.format(len(aid),len(adm_node)))
print(len(note))
print(len(note_aid_pair))

**2-1. Return DB for Masked Literal Pred**

In [6]:
task = 'masked_literal_prediction'
if not os.path.isdir(task):
    os.mkdir(task)
# Build Input
DB = {'train':[],'valid':[],'test':[]}
for sample in note_aid_pair:
    split = np.random.choice(list(DB.keys()),p=[0.8,0.1,0.1])
    if (len(split)>0.1*len(note_aid_pair)) and (split in ['valid', 'test']):
        split = 'train'
    elif (len(split)>0.8*len(note_aid_pair)) and (split in ['train']):
        split = np.random.choice(['valid','test'],p=[0.5,0.5])
    DB[split].append(sample)

# Re-index literals for labeling
literal_id2label = {k:v for (v,k) in enumerate(list(literals.values()))}

for split in DB:
    print('[{}] set size : {}'.format(split, len(DB[split])))
    if not os.path.isdir(os.path.join(task,split)):
        os.mkdir(os.path.join(task,split))
    torch.save([note for (head,note) in DB[split]],'{}/note'.format(os.path.join(task,split)))
    torch.save({'input':[subgraphs[head] for (head,note) in DB[split]],
                'mask':[(~np.isin(np.array(subgraphs[head]),list(literals.keys()))).astype(np.int64).tolist() for (head,note) in DB[split]],
                'label':[list(map(lambda x: literal_id2label[x] if x in literal_id2label else -100,subgraphs[head])) for (head,note) in DB[split]]},
               '{}/kg_norel'.format(os.path.join(task,split)))

[train] set size : 21653
[valid] set size : 2697
[test] set size : 2725


**2-2. Return DB for Literal Bucket Prediction**

In [ ]:
task = 'masked_literal_prediction'
if not os.path.isdir(task):
    os.mkdir(task)
# Build Input
DB = {'train':[],'valid':[],'test':[]}
for sample in note_aid_pair:
    split = np.random.choice(list(DB.keys()),p=[0.8,0.1,0.1])
    if (len(split)>0.1*len(note_aid_pair)) and (split in ['valid', 'test']):
        split = 'train'
    elif (len(split)>0.8*len(note_aid_pair)) and (split in ['train']):
        split = np.random.choice(['valid','test'],p=[0.5,0.5])
    DB[split].append(sample)

# Load Bucket ID
literalID2bucketID = torch.load('literalID2bucketID')

for split in DB:
    print('[{}] set size : {}'.format(split, len(DB[split])))
    if not os.path.isdir(os.path.join(task,split)):
        os.mkdir(os.path.join(task,split))
    torch.save([note for (head,note) in DB[split]],'{}/note'.format(split))
    torch.save({'input':[subgraphs[head] for (head,note) in DB[split]],
                'mask':[(~np.isin(np.array(subgraphs[head]),list(literals.keys()))).astype(np.int64).tolist() for (head,note) in DB[split]],
                'label':[list(map(lambda x: literalID2bucketID[x] if x in literalID2bucketID else -100,subgraphs[head])) for (head,note) in DB[split]]},
               '{}/{}/kg_norel'.format(task,split))

**2-3. Return DB for Contrastive Learning**

In [ ]:
def negative_sampling(input, mask):

task = 'masked_literal_prediction'
if not os.path.isdir(task):
    os.mkdir(task)
# Build Input
DB = {'train':[],'valid':[],'test':[]}
for sample in note_aid_pair:
    split = np.random.choice(list(DB.keys()),p=[0.8,0.1,0.1])
    if (len(split)>0.1*len(note_aid_pair)) and (split in ['valid', 'test']):
        split = 'train'
    elif (len(split)>0.8*len(note_aid_pair)) and (split in ['train']):
        split = np.random.choice(['valid','test'],p=[0.5,0.5])
    DB[split].append(sample)

# Re-index literals for labeling
literal_id2label = {k:v for (v,k) in enumerate(list(literals.values()))}

for split in DB:
    print('[{}] set size : {}'.format(split, len(DB[split])))
    if not os.path.isdir(os.path.join(task,split)):
        os.mkdir(os.path.join(task,split))
    torch.save([note for (head,note) in DB[split]],'{}/note'.format(split))
    torch.save({'input':[subgraphs[head] for (head,note) in DB[split]],
                'mask':[(~np.isin(np.array(subgraphs[head]),list(literals.keys()))).astype(np.int64).tolist() for (head,note) in DB[split]],
                'label':[list(map(lambda x: literal_id2label[x] if x in literal_id2label else -100,subgraphs[head])) for (head,note) in DB[split]]},
               '{}/{}/kg_norel'.format(task,split))

**2-4. Return DB for Literal Regression**

**Supp 1. Save DB in torch.tensor format**

In [ ]:
# Only for DB in tensor form
# Get id sequence of notes

tensorized_subgraphs = torch.LongTensor([subgraphs[x] for x in subgraphs])
print(max_len)
print(len(subgraphs))
print(tensorized_subgraphs[0,:20])
print('Saving...')
torch.save(tensorized_subgraphs,'subgraph_norel')
print('Done')

In [10]:
list(literal_id2label.items())[-1]

(2901117, 9101)